In [3]:
import psycopg2
import xarray as xr
import requests

# Download the NetCDF file
file_url = "https://www.ncei.noaa.gov/data/nclimgrid-daily/access/grids/2000/ncdd-200012-grd-scaled.nc"
file_path = "ncdd-200012-grd-scaled.nc"
response = requests.get(file_url)
print("file downloaded")
with open(file_path, 'wb') as file:
    file.write(response.content)
print("file written")
# Load the dataset from the downloaded file
ds = xr.open_dataset(file_path)

# Extract the latitude and longitude coordinates
latitudes = ds['lat'].values
longitudes = ds['lon'].values

# Combine latitude and longitude into coordinate pairs and convert to standard Python floats
coordinates = [(float(lat), float(lon)) for lat in latitudes for lon in longitudes]

# Close the dataset
ds.close()

# Database connection parameters
db_params = {
    'dbname': 'postgres',
    'user': 'postgres',
    'password': 'Passwordd',
    'host': 'localhost',
    'port': '5432'
}

# Connect to the PostgreSQL database
conn = psycopg2.connect(**db_params)
cur = conn.cursor()

# Create the table if it does not exist
create_table_query = '''
CREATE TABLE IF NOT EXISTS noaa_grid_coords (
    id SERIAL PRIMARY KEY,
    latitude DOUBLE PRECISION,
    longitude DOUBLE PRECISION
);
'''
cur.execute(create_table_query)
print("table created")
# Insert coordinates into the table
insert_query = '''
INSERT INTO noaa_grid_coords (latitude, longitude) VALUES (%s, %s)
'''
cur.executemany(insert_query, coordinates)
print("inserted")
# Commit the transaction and close the connection
conn.commit()
cur.close()
conn.close()

file downloaded
file written
table created
inserted
